# Семинар - Сверточные нейронные сети (Convolution neural network)

In [ ]:
import random
import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.datasets import cifar10

## Загрузим данные

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
print('В трейне: ', x_train.shape[0], 'объектов, в тесте: ', x_test.shape[0])

In [ ]:
num_classes = 10 #
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# Сгенерируем случаные примеры для каждого класса
fig = plt.figure(figsize=(8,3))
for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(y_train[:]==i)[0]
    features_idx = x_train[idx,::]
    img_num = np.random.randint(features_idx.shape[0])
    im = np.transpose(features_idx[img_num,::],(0,1,2))
    ax.set_title(class_names[i])
    plt.imshow(im)
plt.show()

In [ ]:
x_train, x_val, y_train, y_val =  train_test_split(x_train, y_train, test_size=3000, train_size=47000)

In [ ]:
print('Training data shape: ', x_train.shape)
print('Validation data shape: ', x_val.shape)
print('Test data shape: ', x_test.shape)

## Импорт библиотек и подготовка данных

In [ ]:
from keras.models import Sequential # Модель, где все слои соединены друг с другом
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten 
from keras.utils import np_utils
from keras.constraints import maxnorm
from keras.optimizers import Adam

In [ ]:
np.random.seed(18)

x_train = x_train.astype('float32')
x_val= x_val.astype('float32')
x_test = x_test.astype('float32')

__Масштабируем данные__

In [ ]:
# Ваш код здесь

In [ ]:
# Закодируем целевую переменную
y_train_ohe = # Ваш код здесь
y_val_ohe = # Ваш код здесь
y_test_ohe = # Ваш код здесь

## Создание CNN
__Baseline:__

- Convolutional input layer, 16 feature maps with a size of 5×5, a rectifier activation function and a weight constraint of max norm set to 3.
- Max Pool layer with size 2×2.
- Convolutional layer, 16 feature maps with a size of 5×5, a rectifier activation function and a weight constraint of max norm set to 3.
- Max Pool layer with size 2×2.
- Flatten layer.
- Fully connected layer with 512 units and a rectifier activation function.
- Dropout set to 50%.
- Fully connected layer with 128 units and a rectifier activation function.
- Dropout set to 50%.
- Fully connected output layer with 10 units and a softmax activation function.

In [ ]:
lrate = 1e-5

In [ ]:
# Создаем последовательную модель
model = Sequential()

# Добавляем слои
# Ваш код здесь

# Компилируем модель
adam = Adam(lr=lrate)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model.summary())

## Обучение CNN

In [ ]:
epochs = 50
batch_size = 32

In [ ]:
# Обучаем модель! 
history = model.fit(x_train, y_train_ohe,
              epochs=epochs, batch_size=batch_size,
              validation_data=(x_val, y_val_ohe),
              shuffle=True)

In [ ]:
plt.plot(history.history['val_acc'], label='Test')
plt.plot(history.history['acc'], label='Train')
plt.xlabel('Epoch number')
plt.ylabel('Acc')
plt.legend()
plt.show()

In [ ]:
accuracy_score(y_test, model.predict_classes(x_test))

## Сохранение и загрузка моделей

#### Сохранение модели

In [ ]:
with open('cnn.json', 'w') as model_file:
            model_file.write(model.to_json())

#### Сохранение весов

In [ ]:
model.save_weights('cnn.h5')

#### Загрузка модели

In [ ]:
from keras.models import model_from_json

json_file = open("cnn.json", "r")
loaded_model_json = json_file.read()
json_file.close()
# Создаем модель на основе загруженных данных
loaded_model = model_from_json(loaded_model_json)

In [ ]:
# Проверим
loaded_model.summary()

#### Загрузим обученные веса

In [ ]:
loaded_model.load_weights("cnn.h5")

## Предсказания

In [ ]:
predict = loaded_model.predict(x_test)
predicted_class = model.predict_classes(x_test)
print('Accuracy score: ', accuracy_score(y_test, predicted_class ))

In [ ]:
# Проверим случаный семпл:
num_pred = np.random.randint(0,999)

print('True class: ' + class_names[np.where(y_test_ohe[num_pred]==1)[0][0]])
print('Predicted class: ' + class_names[predicted_class[num_pred]])


fig, (ax1, ax2) = plt.subplots(
    nrows=1, ncols=2,
    figsize=(15, 5)
)

ax1.imshow(x_test[num_pred])
pd.Series(predict[num_pred], index=class_names).plot(kind='barh', ax=ax2, figsize=(12,4))
plt.show()

## Как "видит" мир нейронная сеть

In [ ]:
from keras import Model

In [ ]:
# Функция для отрисовки фильтров
def plot_conv_weights(model, layer_name):
    W = model.get_layer(name=layer_name).get_weights()[0]
    if len(W.shape) == 4:
        W = np.squeeze(W)
        W = W.reshape((W.shape[0], W.shape[1], W.shape[2]*W.shape[3]))
        
        fig, axs = plt.subplots(2,8, figsize=(16,4))

        fig.subplots_adjust(hspace = .2, wspace=0.5)
        axs = axs.ravel()
        for i in range(16):
            axs[i].imshow(W[:,:,i], cmap='binary')
            axs[i].set_title(str(i+1))

In [ ]:
plot_conv_weights(loaded_model, 'conv2d_13')
plot_conv_weights(loaded_model, 'conv2d_14')

In [ ]:
#Выберем слой, для визуализации:

layer = 0

activation_model = Model(inputs=loaded_model.input, 
                         outputs=loaded_model.layers[layer].output)
activation_model.summary()

test_img = np.expand_dims(x_test[num_pred], axis=0)
activation = activation_model.predict(test_img)

print(activation.shape)


In [ ]:
plt.title(class_names[np.where(y_test_ohe[num_pred]==1)[0][0]])
plt.imshow(x_test[num_pred])

fig, axs = plt.subplots(2,8, figsize=(16,4))
fig.subplots_adjust(hspace = .2, wspace=0.5)
axs = axs.ravel()
for i in range(16):
    axs[i].imshow(activation[0, :, :, i], cmap='binary')
    axs[i].set_title(str(i+1))